In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
import pyanalib.pandas_helpers as ph
from makedf.util import *

import kinematics
import gump_cuts as gc

In [ ]:
PLOTDIR = "/Users/gputnam/Work/osc/cafpyana/evtsel-plots/"

DOSAVE = True

import os
os.makedirs(PLOTDIR, exist_ok=True)
os.makedirs(PLOTDIR + "/png", exist_ok=True)
os.makedirs(PLOTDIR + "/pdf", exist_ok=True)

In [ ]:
DETECTOR = "ICARUS"

In [ ]:
DETNAME = DETECTOR.replace(" ", "-")

In [ ]:
if DETECTOR == "ICARUS":
    FILE = "/Users/gputnam/Work/osc/cafpyana/analysis_village/gump/ICARUS_SpringMC_Dev_slimwgt.df"
    OFFBEAMFILE = "/Users/gputnam/Work/osc/cafpyana/analysis_village/gump/Run2_BNBoff_uncalo_prescaled.df"
elif DETECTOR == "SBND": 
    FILE = "/Users/gputnam/Work/osc/cafpyana/analysis_village/gump/SBND_SpringMC_5000.df"
    OFFBEAMFILE = "/Users/gputnam/Work/osc/cafpyana/analysis_village/gump/SBND_SpringBNBOffData_5000.df"
elif DETECTOR == "SBND SPINE":
    FILE = "/Users/gputnam/Work/osc/cafpyana/analysis_village/gump/SBND_SPINE_SpringMC.df"
    OFFBEAMFILE = "/Users/gputnam/Work/osc/cafpyana/analysis_village/gump/SBND_SPINE_SpringBNBOffData.df"

In [ ]:
import h5py

def read_dfs(file, key):
    with h5py.File(file, "r") as f:
        keys = [k for k in f.keys() if k.startswith(key)]
        return pd.concat([pd.read_hdf(file, k) for k in keys])

In [ ]:
def load_data(file, nfiles=-1):
    """Load event, header, and mcnu data from HDF file."""

    if nfiles < 0:
        with h5py.File(file, "r") as f:
            nfiles = len([k for k in f.keys() if k.startswith("hdr")])

    for s in range(nfiles):
        print("df index:"+str(s))
        if "SPINE" not in DETECTOR:
            df_evt = pd.read_hdf(file, "evt_"+str(s))
        else:
            df_evt = pd.read_hdf(file, "sevt_"+str(s))
            
        df_hdr = pd.read_hdf(file, "hdr_"+str(s))
        df_mcnu = pd.read_hdf(file, "mcnu_"+str(s))

        matchdf = df_evt.copy()
        matchdf.columns = pd.MultiIndex.from_tuples([(col, '') for col in matchdf.columns])
        df_evt = ph.multicol_merge(matchdf.reset_index(), df_mcnu.reset_index(),
                                    left_on=[("__ntuple", ""), ("entry", ""), ("tmatch_idx", "")],
                                    right_on=[("__ntuple", ""), ("entry", ""), ("rec.mc.nu..index", "")],
                                    how="left") ## -- save all sllices

        if s == 0:
            res_df_evt = df_evt
            res_df_hdr = df_hdr
            res_df_nu = df_mcnu
        else:
            res_df_evt = pd.concat([res_df_evt, df_evt])
            res_df_hdr = pd.concat([res_df_hdr, df_hdr])
            res_df_nu = pd.concat([res_df_nu, df_mcnu])

        del df_evt
        del df_hdr

    return res_df_evt, res_df_hdr, res_df_nu

In [ ]:
df, hdr, mcnu = load_data(FILE)

In [ ]:
hdr # .pot.groupby(level=[0, 1]).first()

In [ ]:
df.columns = [c[0] for c in df.columns]
df["offbeam"] = False

In [ ]:
dfOFF, _,_ = load_data(OFFBEAMFILE)

In [ ]:
dfOFF.columns = [c[0] for c in dfOFF.columns]
dfOFF["offbeam"] = True

In [ ]:
crtOFF = read_dfs(OFFBEAMFILE, "crt")

In [ ]:
_ = plt.hist(crtOFF.time, bins=np.linspace(-2.5, 2.5, 51))

plt.axvline([-1], color="red")
plt.axvline([1.8], color="red")
plt.title("Off Beam BNBMajority Data")

if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-beam_off_crt_tophat.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-beam_off_crt_tophat.png" % DETNAME, bbox_inches="tight")

In [ ]:
crthit = ((crtOFF.time > -1) & (crtOFF.time < 1.8)).groupby(level=[0, 1]).any()
crthit.name = "crthit"

crthit.mean()
dfOFF = dfOFF.join(crthit, on=["__ntuple", "entry"])

In [ ]:
crtMC = read_dfs(FILE, "crt")

In [ ]:
_ = plt.hist(crtMC.time, bins=np.linspace(-2.5, 2.5, 51))

plt.axvline([-1], color="red")
plt.axvline([1.8], color="red")
plt.title("Overlay (Off-Beam Minbias + MC)")

if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-overlay_crt_tophat.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-overlay_crt_tophat.png" % DETNAME, bbox_inches="tight")

In [ ]:
crthitMC = ((crtMC.time > -1) & (crtMC.time < 1.8)).groupby(level=[0, 1]).any()
crthitMC.name = "crthit"

crthitMC.mean()

if "crthit" not in df.columns:
    df = df.join(crthitMC, on=["__ntuple", "entry"])

In [ ]:
df = pd.concat([df, dfOFF])

In [ ]:
# false positive CRT hit rate

crthit_test = ((crtOFF.time > -0.6 + 10) & (crtOFF.time < 1.7 + 10)).groupby(level=[0, 1]).any()
crthit_test.name = "crthit"

crthit_false_positive_rate = crthit_test.mean()
crthit_false_positive_rate

In [ ]:
def scale_pot(df, df_hdr, desired_pot):
    """Scale DataFrame by desired POT."""
    pot = sum(df_hdr.pot.tolist())
    print(f"POT: {pot}\nScaling to: {desired_pot}")
    scale = desired_pot / pot
    df['glob_scale'] = scale
    return pot, scale

In [ ]:
if "ICARUS" in DETECTOR:
    GOAL_POT = 5e20
    N_GATES_ON_PER_5e12POT = 1.3886218026202426
    ngates_OFF = read_dfs(OFFBEAMFILE, "trig").gate_delta.sum()*(1-1/20.)
elif "SBND" in DETECTOR:
    GOAL_POT = 1e20
    N_GATES_ON_PER_5e12POT = 1.05104
    ngates_OFF = read_dfs(OFFBEAMFILE, "hdr").noffbeambnb.sum()

In [ ]:
scale_pot(df, hdr, GOAL_POT)

In [ ]:
glob_scale = df['glob_scale'].to_numpy()[0]
glob_scale

In [ ]:
df.loc[df.offbeam, "glob_scale"] = (GOAL_POT/5e12)*(N_GATES_ON_PER_5e12POT / ngates_OFF)
(GOAL_POT/5e12)*(N_GATES_ON_PER_5e12POT / ngates_OFF)

In [ ]:
mode_list = [0, 10, 1, 2, 3]
mode_labels = ['QE', 'MEC', 'RES', 'SIS/DIS', 'COH', "other"]

def breakdown_mode(var, df):
    """Break down variable by interaction mode."""
    ret = [var[df.genie_mode == i] for i in mode_list]
    ret.append(var[sum([df.genie_mode == i for i in mode_list]) == 0])
    return ret

In [ ]:
FONTSIZE = 14
HAWKS_COLORS = ["#315031", "#d54c28", "#1e3f54", "#c89648", "#43140b", "#95af8b"]

def add_style(ax, xlabel, title="", det=DETECTOR):
    ax.tick_params(axis='both', which='both', direction='in', length=6, width=1.5, labelsize=FONTSIZE, top=True, right=True)
    for spine in ax.spines.values():
        spine.set_linewidth(1.5)
    ax.set_xlabel(xlabel, fontsize=FONTSIZE, fontweight='bold')
    ax.set_ylabel('Events', fontsize=FONTSIZE, fontweight='bold')
    det = det.replace(" ", "-")
    ax.set_title(f"$\\bf{{{det}}}$  {title}", fontsize=FONTSIZE+2)
    ax.legend(fontsize=FONTSIZE)

In [ ]:
FV = gc.slcfv_cut(df, DETECTOR.split(" ")[0]) & gc.mufv_cut(df, DETECTOR.split(" ")[0]) & gc.pfv_cut(df, DETECTOR.split(" ")[0]) 

In [ ]:
cut = FV

In [ ]:
var = "nu_score"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0,1,21), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "$\\nu$ Score", "Contained")

if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-nu_score_contained.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-nu_score_contained.png" % DETNAME, bbox_inches="tight")

In [ ]:
var = "crlongtrkdiry"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(-1,1,21), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "CRLongTrkDirY", "Fiducial")

if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-crlongtrkdiry_contained.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-crlongtrkdiry_contained.png" % DETNAME, bbox_inches="tight")

In [ ]:
def cosmic_cut(df):
    if "SPINE" not in DETECTOR:
        return df.nu_score > 0.4
    else:
        return df.is_time_contained
        
cut = FV & cosmic_cut(df)

In [ ]:
var = "mu_len"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 500,21), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "Muon Length [cm]", title="Cosmic Rej.")

if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-muon_length_cosmicrej.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-muon_length_cosmicrej.png" % DETNAME, bbox_inches="tight")

In [ ]:
var = "other_shw_length"

pvar = breakdown_mode(df.loc[cut, var].fillna(-1), df[cut])
n, bins, _ = plt.hist(pvar, bins=np.array([-10] + list(np.linspace(0, 200, 21))), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])
plt.yscale("log")
add_style(plt.gca(), "Maximum Shower Length [cm]", title="Cosmic Rej.")


if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-shwlen_cosmicrej.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-shwlen_cosmicrej.png" % DETNAME, bbox_inches="tight")

In [ ]:
var = "other_trk_length"

pvar = breakdown_mode(df.loc[cut, var].fillna(-1), df[cut])
n, bins, _ = plt.hist(pvar, bins=np.array([-10] + list(np.linspace(0, 200, 21))), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])
plt.yscale("log")
add_style(plt.gca(), "Maximum Track Length [cm]", title="Cosmic Rej.")


if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-thirdtrklen_cosmicrej.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-thirdtrklen_cosmicrej.png" % DETNAME, bbox_inches="tight")

In [ ]:
def twoprong_cut(df):
    return np.isnan(df.other_shw_length) & np.isnan(df.other_trk_length)

In [ ]:
cut = FV & cosmic_cut(df) & twoprong_cut(df)

In [ ]:
var = "mu_len"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 500,21), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "Muon Length [cm]", title="Two Prong")


if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-muon_length_twoprong.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-muon_length_twoprong.png" % DETNAME, bbox_inches="tight")

In [ ]:
var = "mu_chi2_of_mu_cand"

if DETECTOR == "ICARUS":
    bins = np.linspace(0, 60,21)
elif DETECTOR == "SBND SPINE":
    bins = np.linspace(0, 1, 21)

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "Muon Candidate $\\mu$-like PID", title="Two Prong")

if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-mucand_chi2u_twoprong.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-mucand_chi2u_twoprong.png" % DETNAME, bbox_inches="tight")

In [ ]:
var = "prot_chi2_of_mu_cand"

if DETECTOR == "ICARUS":
    bins = np.linspace(0, 400,21)
elif DETECTOR == "SBND SPINE":
    bins = np.linspace(0, 1, 21)

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "Muon Candidate $p$-like PID", title="Two Prong")

if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-mucand_chi2p_twoprong.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-mucand_chi2p_twoprong.png" % DETNAME, bbox_inches="tight")

In [ ]:
var = "mu_chi2_of_prot_cand"

if DETECTOR == "ICARUS":
    bins = np.linspace(0, 60,21)
elif DETECTOR == "SBND SPINE":
    bins = np.linspace(0, 1, 21)

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "Proton Candidate $\\mu$-like PID", title="Two Prong")


if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-pcand_chi2u_twoprong.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-pcand_chi2u_twoprong.png" % DETNAME, bbox_inches="tight")

In [ ]:
var = "prot_chi2_of_prot_cand"

if DETECTOR == "ICARUS":
    bins = np.linspace(0, 60,21)
elif DETECTOR == "SBND SPINE":
    bins = np.linspace(0, 1, 21)

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])


add_style(plt.gca(), "Proton Candidate $p$-like PID", title="Two Prong")


if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-pcand_chi2p_twoprong.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-pcand_chi2p_twoprong.png" % DETNAME, bbox_inches="tight")

In [ ]:
def pid_cut(df):
    if "SPINE" not in DETECTOR:
        return gc.pid_cut_df(df)
    else:
        return (df.prot_chi2_of_prot_cand > 0.6) & (df.mu_chi2_of_mu_cand > 0.6) & (df.mu_len > 25) & (df.p_len > 5)

In [ ]:
cut = FV & cosmic_cut(df) & twoprong_cut(df) & pid_cut(df) # & (~df.crthit)

# cut = FV & twoprong_cut(df) & gc.pid_cut_df(df)

In [ ]:
var = "mu_len"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 600, 11), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "Muon Length [cm]", title="PID")


if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-muon_length_pid.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-muon_length_pid.png" % DETNAME, bbox_inches="tight")

In [ ]:
var = "mu_len"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 200, 11), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "Muon Length [cm]", title="PID")


if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-muon_length_zoom_pid.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-muon_length_zoom_pid.png" % DETNAME, bbox_inches="tight")

In [ ]:
var = "p_len"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 50, 11), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "Proton Length [cm]", title="PID")

if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-proton_length_pid.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-proton_length_pid.png" % DETNAME, bbox_inches="tight")

In [ ]:
var = "del_p"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 1, 11), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "$\\delta p$ [GeV]", title="PID")


if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-delp_pid.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-delp_pid.png" % DETNAME, bbox_inches="tight")

In [ ]:
var = "nu_E_calo"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 2, 16), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "$E_\\nu^\\mathrm{calo}$ [GeV]", title="PID")

if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-ecalo_pid.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-ecalo_pid.png" % DETNAME, bbox_inches="tight")

In [ ]:
if "ICARUS" in DETECTOR:
    cut = FV & cosmic_cut(df) & twoprong_cut(df) & pid_cut(df) & (~df.crthit)

In [ ]:
var = "del_p"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 1, 11), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "$\\delta p$ [GeV]", title="PID + CRT Hit Veto")


if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-delp_crthitveto.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-delp_crthitveto.png" % DETNAME, bbox_inches="tight")

In [ ]:
cut = FV & cosmic_cut(df) & twoprong_cut(df) & pid_cut(df) & (df.del_p < 0.25) # & (~df.crthit)

In [ ]:
var = "nu_E_calo"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 2, 16), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "$E_\\nu^\\mathrm{calo}$ [GeV]", title="PID + $\\delta_p < 0.25$ GeV")


if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-ecalo_lowdelp.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-ecalo_lowdelp.png" % DETNAME, bbox_inches="tight")

In [ ]:
var = "p_len"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 100, 11), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "Proton Length", title="PID + $\\delta_p < 0.25$ GeV")


if DOSAVE:
    plt.savefig(PLOTDIR + "pdf/%s-proton_length_lowdelp.pdf" % DETNAME, bbox_inches="tight")
    plt.savefig(PLOTDIR + "png/%s-proton_length_lowdelp.png" % DETNAME, bbox_inches="tight")

In [ ]:
cut = FV & cosmic_cut(df) & twoprong_cut(df) & ~np.isnan(df.genie_mode)

In [ ]:
df.columns

In [ ]:
var = "mu_chi2_of_prot_cand"

if DETECTOR == "ICARUS":
    bins = np.linspace(0, 100,21)
elif DETECTOR == "SBND SPINE":
    bins = np.linspace(0, 1, 21)

n, bins, _ = plt.hist(df.loc[cut, var], bins=bins, stacked=True, label="CV",
                    color="tab:blue", weights=df.loc[cut, "glob_scale"], histtype="step", linewidth=3)

n, bins, _ = plt.hist(df.loc[cut, var.replace("chi2", "chi2smear")], bins=bins, stacked=True, label="Smear +5%",
                    color="tab:orange", weights=df.loc[cut, "glob_scale"], histtype="step", linewidth=3)

n, bins, _ = plt.hist(df.loc[cut, var.replace("chi2", "chi2hi")], bins=bins, stacked=True, label="Gain +1%",
                    color="tab:green", weights=df.loc[cut, "glob_scale"], histtype="step", linewidth=3)

add_style(plt.gca(), "Proton Candidate $\\mu$-like PID", title="Two Prong, $\\nu$-Only")


# if DOSAVE:
    # plt.savefig(PLOTDIR + "pdf/%s-pcand_chi2u_twoprong.pdf" % DETNAME, bbox_inches="tight")
    # plt.savefig(PLOTDIR + "png/%s-pcand_chi2u_twoprong.png" % DETNAME, bbox_inches="tight")

In [ ]:
df.columns

In [ ]:
df.genie_mode